# Converting AFTER SMPL->BVH

In [ ]:
from pathlib import Path

import numpy as np
from Motion.smpl import load_smpl, smpl_to_bvh_data
from Motion.smpl_utils.utils import bvh
from scipy.spatial.transform import Rotation as R

pkl_file = "ref_Zup.pkl"
smpl_dict = load_smpl(pkl_file)

# Convert back to Y-up for rendering in blender and three.js
rotation = R.from_quat(np.array([ -0.7071068, 0, 0, 0.7071068 ])) # -90 degrees about the x axis
root_rotvec = smpl_dict["smpl_poses"][:, 0]
root_rotvec = (rotation * R.from_rotvec(root_rotvec)).as_rotvec()
bvh_data = smpl_to_bvh_data(smpl_dict, frametime=1 / 30)
root_euler = R.from_rotvec(root_rotvec).as_euler(bvh_data["order"], degrees=True)
bvh_data["rotations"][:, 0] = root_euler

smpl_trans_y_up = np.copy(bvh_data["positions"])
# For the positions, swap Y and Z
smpl_trans_y_up[..., 1] = bvh_data["positions"][..., 2]  # don't forget the minus!!!
smpl_trans_y_up[..., 2] = -bvh_data["positions"][..., 1]
bvh_data["positions"] = smpl_trans_y_up

bvh.save(f"{Path(pkl_file).stem}_after.bvh", bvh_data)

# Converting BEFORE SMPL->BVH

In [ ]:
from glob import glob
from pathlib import Path

import numpy as np
from Motion import BVH
from Motion.smpl import load_smpl, smpl_to_bvh_data
from Motion.smpl_utils.utils import bvh
from scipy.spatial.transform import Rotation as R

pkl_file = "ref_Zup.pkl"
smpl_dict = load_smpl(pkl_file)

# Convert back to Y-up for rendering in blender and three.js
rotation = R.from_quat(np.array([ -0.7071068, 0, 0, 0.7071068 ])) # -90 degrees about the x axis
root_rotvec = smpl_dict["smpl_poses"][:, 0]
root_rotvec = (rotation * R.from_rotvec(root_rotvec)).as_rotvec()
smpl_dict["smpl_poses"][:, 0] = root_rotvec

# For the positions, swap Y and Z
smpl_trans_y_up = np.copy(smpl_dict["smpl_trans"])
smpl_trans_y_up[..., 1] = smpl_dict["smpl_trans"][..., 2]
smpl_trans_y_up[..., 2] = -smpl_dict["smpl_trans"][..., 1] # don't forget the minus!!!
smpl_dict["smpl_trans"] = smpl_trans_y_up

bvh_data = smpl_to_bvh_data(smpl_dict, frametime=1 / 30)
bvh.save(f"{Path(pkl_file).stem}_before.bvh", bvh_data)